# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
2. 워드 임베딩 모델을 학습시킨다.
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다.

=><br>
A:B = C:D 괸계에 대응하는 D를 찾는 게임을 만든다.<br>
ex) A:산, B:바다, C:나무, D:물<br>

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ?]<br>
모델이 예측한 가장 적합한 단어:잠입<br>
당신의 답변과 모델 예측의 유사도:0.34<br>
아쉽네요. 더 생각해보세요.<br>

In [1]:
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [2]:
# 1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from konlpy.tag import Okt

okt = Okt()

In [3]:
f = open('kowiki-20250801-pages-articles-multistream-index.txt', 'r', encoding='utf-8')

preprocessed_text = []

ko_stopwords = r'[a-zA-Z0-9]'

for line in f:
    tokens = okt.morphs(line)
    tokens = [token for token in tokens if not re.search(r'[\s.,:;?!]', token)] # 공백 및 구두점 제거
    tokens = [token for token in tokens if not re.search(ko_stopwords, token)]  # 정규식으로 불용어 제거
    preprocessed_text.append(tokens)

preprocessed_text[:5]

[['위키', '백', '과', '대문'], ['지미', '카터'], ['수학'], ['수학', '상수'], ['도움말', '도움말']]

In [4]:
# 2. 워드 임베딩 모델을 학습시킨다.
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=preprocessed_text,   # corpus
    vector_size =100,                   # 임베딩 벡터 차원
    sg=0,                               # 학습 알고리즘 (CBOW = 0, Skip-gram = 1)
    window=5,                          # 주변 단어 수 (앞뒤로 n개 사용)
    min_count=5                         # 최소 빈도 (corpus에서 5개 미만으로 나타난 빈도에 대해 제거)
)

In [5]:
model.wv.save_word2vec_format('ko_wiki_w2v')

In [6]:
model.wv.index_to_key

['분류',
 '의',
 '(',
 ')',
 '틀',
 '/',
 '-',
 '선수',
 '제',
 '과',
 '한',
 '이',
 '축구',
 '백',
 '에',
 '역',
 '대한민국',
 '주',
 '시',
 '별',
 '대',
 '영화',
 '위키',
 '올림픽',
 '된',
 '로',
 '라',
 '파일',
 '계',
 '아',
 '가',
 '군',
 '세',
 '문서',
 '와',
 '초등학교',
 '목록',
 '스포츠',
 '하',
 '여자',
 '미국',
 '게임',
 '이의',
 '오',
 '구',
 '촌',
 '동음',
 '나',
 '주의',
 '도',
 '스',
 '니',
 '다',
 '일본',
 '리',
 '인',
 '동문',
 '참가',
 '리그',
 '성',
 '즈',
 '한국',
 '대회',
 '지',
 '남자',
 '조',
 '사람',
 '설명',
 '모듈',
 '타',
 '카',
 '사',
 '현',
 '야',
 '국제',
 '텔레비전',
 '자',
 '고',
 '을',
 '작품',
 '고등학교',
 '네',
 '출신',
 '더',
 '관계',
 '설립',
 '음반',
 '대학교',
 '역사',
 '중학교',
 '사건',
 '기',
 '세기',
 '나라',
 '동',
 '기업',
 '코',
 '서',
 '교통',
 '서울',
 '정',
 '감독',
 '세계',
 '키',
 '영국',
 '호',
 '공화국',
 '요',
 '그',
 '학교',
 '상',
 '를',
 '으로',
 '국가대표팀',
 '소',
 '건축물',
 '독일',
 '프랑스',
 '중국',
 '명단',
 '원',
 '히',
 '드라마',
 '단체',
 '미',
 '모',
 '대학',
 '선거',
 '신',
 '트',
 '관',
 '정보',
 '차',
 '드',
 '도시',
 '기원전',
 '배우',
 '우',
 '선',
 '노',
 '데',
 '수',
 '축구단',
 '섬',
 '토론',
 '전쟁',
 '에서',
 '철도',
 '동계',
 '어',
 '나라자료',

In [14]:
# 3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
import random

A = random.choice(model.wv.index_to_key)

B = random.choice([word for word in model.wv.index_to_key if word != A and model.wv.similarity(A, word) >= 0.8])
# 4. A, B와 대응되는 C를 추출한다.
# 비례식 A: B = C:? 일 때 C를 추출하는 방법 A+C-B
C = model.wv.most_similar(positive=[B[0], A], negative=[B[1]], topn=1)[0][0]
print(f'관계 [{A} : {B} = {C} : ? ]')

관계 [찰름 : 프란코 = 끼앗군 : ? ]


In [15]:
# 5. D를 추출한다.
# D는 A, B, C와 유사도가 가장 높은 단어로 추출
D = input('당신의 답변을 입력하세요: ')
predicted_D = model.wv.most_similar(positive=[B, C], negative=[A], topn=1)[0][0]
sim = model.wv.similarity(predicted_D, C)
res = model.wv.n_similarity([A, B[0]], [C, D])
preddict = model.wv.similarity(D, predicted_D)  # 입력한 단어 D와 모델이 예측한 단어 D 간의 유사도


print(f'단어 {C}와 단어 {D}간에 유사도: {sim}')
print(f'단어 {A}, {B}, {C}, {predicted_D}간에 유사도: {res}')
print(f'내가 입력한 단어 {D}와 모델이 예측한 단어 {predicted_D} 간에 유사도: {sim}')
print(f'모델이 예측한 가장 적합한 단어: {predicted_D}')
if sim >= 0.95:
    print('와우 당신은 추론의 제왕!!!!')
elif 0.7 < sim < 0.95:
    print('좋아요! 당신은 추론의 왕!')
elif 0.5 < sim <= 0.7:
    print('괜찮아요! 당신은 평범한 사람입니다!!!!')
elif 0.3 < sim <= 0.5:
    print('아... 아쉬워요 다음게임을 노리세요!!!!!')
else:
    print('당신에게는 기회를 줄 가치가 없군요!!!!!')

단어 끼앗군와 단어 난쟁이간에 유사도: 0.7725209593772888
단어 찰름, 프란코, 끼앗군, 고물자리간에 유사도: 0.21644803881645203
내가 입력한 단어 난쟁이와 모델이 예측한 단어 고물자리 간에 유사도: 0.7725209593772888
모델이 예측한 가장 적합한 단어: 고물자리
좋아요! 당신은 추론의 왕!
